## DNN Retry Max
Included the minimum, as well as took out the data from 2020-2021, 2021-2022.

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from statistics import mean
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
from statistics import mean
import math

FUTURE_PREDICTION = 4

def findmax(x):
    maximum = 0
    for number in x:
        maximum = max(x)
    return maximum

def findmean(x):
    m = 0
    m = mean(x)
    return m

def findmin(x):
    m = 0
    m = min(x)
    return m

def CreateMean(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Mean'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            mean = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Mean'])
            mean = findmean(list)
            dataframe.at[x,f'{title}_Mean'] = mean
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMax(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Max'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            maximum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Max'])
            maximum = findmax(list)
            dataframe.at[x,f'{title}_Max'] = maximum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

def CreateMin(dataframe,title,city):

    dataframe = dataframe.iloc[522:3924:1]
    dataframe.rename(columns = {f'{city}' : f'{title}_Min'}, inplace = True)
    dataframe = dataframe.reset_index(drop=True)
    for x in dataframe.index:
        if x % 7 == 6:
            list = []
            minimum = 0
            for y in range(x-6,x+1,1):
                list.append(dataframe.iloc[y][f'{title}_Min'])
            minimum = findmin(list)
            dataframe.at[x,f'{title}_Min'] = minimum
    for x in dataframe.index:
        if x % 7 != 6:
            dataframe.drop(index= x, inplace=True)
    dataframe = dataframe.reset_index(drop=True)
    return dataframe
    
def DataPreprocess(city):
    virus_df = pd.read_csv('Data/virus.csv')
    virus_df['Date'] = pd.to_datetime(virus_df['Date'])
    virus_df['year'] = virus_df['Date'].apply(lambda date: date.year)
    virus_df['month'] = virus_df['Date'].apply(lambda date: date.month)
    virus_df_new = virus_df[['Date','month','year',f'{city}']]
    virus_df_new.rename(columns = {f'{city}' : f'Virus_Count_{city}'}, inplace = True)

    temp_df = pd.read_csv(f'Data/{city}_temp.csv')
    pressure_df = pd.read_csv(f'Data/{city}_pressure.csv')
    hours_df = pd.read_csv(f'Data/{city}_hours.csv')
    rainfall_df = pd.read_csv(f'Data/{city}_rainfall.csv')

    temp_df['Date'] = pd.to_datetime(temp_df['Date'])
    temp_df['year'] = temp_df['Date'].apply(lambda date: date.year)
    temp_df['month'] = temp_df['Date'].apply(lambda date: date.month)

    pressure_df['Date'] = pd.to_datetime(pressure_df['Date'])
    pressure_df['year'] = pressure_df['Date'].apply(lambda date: date.year)
    pressure_df['month'] = pressure_df['Date'].apply(lambda date: date.month)
    pressure_df = pressure_df.sort_values(by="Date")
    pressure_df = pressure_df.reset_index(drop=True)

    hours_df['Date'] = pd.to_datetime(hours_df['Date'])
    hours_df['year'] = hours_df['Date'].apply(lambda date: date.year)
    hours_df['month'] = hours_df['Date'].apply(lambda date: date.month)
    hours_df = hours_df.sort_values(by="Date")
    hours_df = hours_df.reset_index(drop=True)

    rainfall_df['Date'] = pd.to_datetime(rainfall_df['Date'])
    rainfall_df['year'] = rainfall_df['Date'].apply(lambda date: date.year)
    rainfall_df['month'] = rainfall_df['Date'].apply(lambda date: date.month)
    rainfall_df = rainfall_df.sort_values(by="Date")
    rainfall_df = rainfall_df.reset_index(drop=True)

    list = []
    for x in temp_df.index:
        t = temp_df.iloc[x][f'{city}']
        svp = 6.1078*10**(7.5*t/(t+237.3))
        e = pressure_df.iloc[x][f'{city}']
        h = e/svp*100
        list.append(h)
        
    humidity_df = pd.DataFrame(data=list)
    humidity_df.rename(columns = {0:f'{city}'}, inplace = True)
    
    max_temp_df = CreateMax(temp_df, 'Temperature',f'{city}')
    max_pressure_df = CreateMax(pressure_df, 'Pressure',f'{city}')
    max_hours_df = CreateMax(hours_df, 'Hours',f'{city}')
    max_rainfall_df = CreateMax(rainfall_df, 'Rainfall',f'{city}')
    max_humidity_df = CreateMax(humidity_df, 'Humidity',f'{city}')


    df = pd.concat([virus_df_new,max_temp_df,max_pressure_df,max_hours_df,max_rainfall_df,max_humidity_df], axis=1)
    
    df = df.loc[:,~df.columns.duplicated()]
    
    df['VirusCRM'] = df['Virus_Count_Tokyo'].rolling(14, center = True).mean()

    for i in df.index:
        if i == 0:
            df.at[i,'Gradient'] = 1
        else:
            if df.iloc[i-1]['VirusCRM'] == 0:
                if df.at[i,'VirusCDM'] > 0:
                    df.at[i,'Gradient'] = 1
                else:
                    df.at[i,'Gradient'] = 0
            else:
                grd = df.iloc[i]['VirusCRM']/(df.iloc[i-1]['VirusCRM'])
                df.at[i,'Gradient'] = grd
    
    for i in df.index:
        if (df.iloc[i]['Gradient'] >= 1.1) and (df.iloc[i+1]['Gradient'] >= 1.1) and (df.iloc[i+2]['Gradient'] >= 1.1):
            df.at[i,'Spreading'] = 1
        else:
            df.at[i,'Spreading'] = 0
    df = df.astype({'Spreading':'int8'})

    df_virus_count = df[f'Virus_Count_{city}']
    df.drop(columns=['VirusCRM',f'Virus_Count_{city}','year','Gradient'], axis=1, inplace=True)
    df_labels = df[['Date']]
    FUTURE_PREDICTION = 4
    for i in range(FUTURE_PREDICTION):
        df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)
    temp = df[["Date", "Spreading"]]
    for i in range(FUTURE_PREDICTION):
        target_column = f"T+{(i+1)*7}"
        df_labels = df_labels.merge(temp, 
                                how ="left",
                                left_on = (target_column),
                                right_on=("Date"),
                                suffixes = ("","_y")
        )
        df_labels.rename(columns={"Spreading":f"{target_column}_Spreading"},inplace=True)
    df_labels.drop(columns=["Date_y"],inplace=True)

    for i in range(FUTURE_PREDICTION):
        df_labels.drop(columns =[f"T+{(i+1)*7}"],inplace=True,axis=1 )

    df = df.merge(df_labels,
             how='left',
             on = ["Date"])
    df.dropna(inplace=True)

    X = df.drop(columns=['T+7_Spreading','T+14_Spreading','T+21_Spreading','T+28_Spreading'],axis=1)
    y1 = df['T+21_Spreading']
    y2 = df['T+7_Spreading']
    y3 = df['T+14_Spreading']
    y4 = df['T+28_Spreading']

    return X, y1, y2, y3, y4, df, df_virus_count

In [3]:
X, y1, y2, y3, y4, df, df_virus_count = DataPreprocess('Tokyo')

/Users/miyanishio/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/k5/3yc13bs57f7637937yjlstth0000gn/T/ipykernel_10325/3620082658.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels[f"T+{(i+1)*7}"] = df_labels["Date"] + pd.DateOffset(days = (i+1)*7)


In [4]:
X = X.drop(columns=['Date'])

In [5]:
X = X[0:400] #until 2020 05-04
y1 = y1[0:400]
y2 = y2[0:400]
y3 = y2[0:400]
y4 = y2[0:400]

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
scaler1 = StandardScaler()
X_train, X_test = X[0:280], X[280:400]
y1_train, y1_test = y1[0:280], y1[280:400]

In [7]:
scaled_X_train = scaler1.fit_transform(X_train)
scaled_X_test = scaler1.transform(X_test)

In [8]:
model = Sequential()

model.add(Dense(12,activation="sigmoid"))
model.add(Dense(6,activation="sigmoid"))
model.add(Dense(3,activation="sigmoid"))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

2022-12-14 12:24:33.329739: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
early_stop = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=50)

In [10]:
model.fit(scaled_X_train,y1_train,epochs=400,validation_data=(scaled_X_test,y1_test),callbacks=[early_stop])

Epoch 1/400
9/9 [==============================] - 2s 47ms/step - loss: 0.7544 - accuracy: 0.1643 - val_loss: 0.7549 - val_accuracy: 0.1000
Epoch 2/400
9/9 [==============================] - 0s 6ms/step - loss: 0.7400 - accuracy: 0.1643 - val_loss: 0.7416 - val_accuracy: 0.1000
Epoch 3/400
9/9 [==============================] - 0s 7ms/step - loss: 0.7292 - accuracy: 0.1643 - val_loss: 0.7296 - val_accuracy: 0.1000
Epoch 4/400
9/9 [==============================] - 0s 6ms/step - loss: 0.7195 - accuracy: 0.1643 - val_loss: 0.7186 - val_accuracy: 0.1000
Epoch 5/400
9/9 [==============================] - 0s 7ms/step - loss: 0.7103 - accuracy: 0.1643 - val_loss: 0.7078 - val_accuracy: 0.1000
Epoch 6/400
9/9 [==============================] - 0s 11ms/step - loss: 0.7015 - accuracy: 0.1643 - val_loss: 0.6979 - val_accuracy: 0.1083
Epoch 7/400
9/9 [==============================] - 0s 8ms/step - loss: 0.6931 - accuracy: 0.5071 - val_loss: 0.6877 - val_accuracy: 0.9000
Epoch 8/400
9/9 [========

9/9 [==============================] - 0s 7ms/step - loss: 0.4496 - accuracy: 0.8357 - val_loss: 0.3681 - val_accuracy: 0.9000
Epoch 60/400
9/9 [==============================] - 0s 5ms/step - loss: 0.4482 - accuracy: 0.8357 - val_loss: 0.3658 - val_accuracy: 0.9000
Epoch 61/400
9/9 [==============================] - 0s 6ms/step - loss: 0.4468 - accuracy: 0.8357 - val_loss: 0.3635 - val_accuracy: 0.9000
Epoch 62/400
9/9 [==============================] - 0s 11ms/step - loss: 0.4454 - accuracy: 0.8357 - val_loss: 0.3610 - val_accuracy: 0.9000
Epoch 63/400
9/9 [==============================] - 0s 9ms/step - loss: 0.4441 - accuracy: 0.8357 - val_loss: 0.3590 - val_accuracy: 0.9000
Epoch 64/400
9/9 [==============================] - 0s 5ms/step - loss: 0.4429 - accuracy: 0.8357 - val_loss: 0.3570 - val_accuracy: 0.9000
Epoch 65/400
9/9 [==============================] - 0s 6ms/step - loss: 0.4416 - accuracy: 0.8357 - val_loss: 0.3551 - val_accuracy: 0.9000
Epoch 66/400
9/9 [==============

9/9 [==============================] - 0s 8ms/step - loss: 0.3729 - accuracy: 0.8357 - val_loss: 0.3000 - val_accuracy: 0.9000
Epoch 118/400
9/9 [==============================] - 0s 6ms/step - loss: 0.3715 - accuracy: 0.8357 - val_loss: 0.2992 - val_accuracy: 0.9000
Epoch 119/400
9/9 [==============================] - 0s 6ms/step - loss: 0.3699 - accuracy: 0.8357 - val_loss: 0.2982 - val_accuracy: 0.9000
Epoch 120/400
9/9 [==============================] - 0s 7ms/step - loss: 0.3682 - accuracy: 0.8357 - val_loss: 0.2973 - val_accuracy: 0.9000
Epoch 121/400
9/9 [==============================] - 0s 5ms/step - loss: 0.3667 - accuracy: 0.8357 - val_loss: 0.2963 - val_accuracy: 0.9000
Epoch 122/400
9/9 [==============================] - 0s 5ms/step - loss: 0.3651 - accuracy: 0.8357 - val_loss: 0.2953 - val_accuracy: 0.9000
Epoch 123/400
9/9 [==============================] - 0s 6ms/step - loss: 0.3635 - accuracy: 0.8357 - val_loss: 0.2944 - val_accuracy: 0.9000
Epoch 124/400
9/9 [========

9/9 [==============================] - 0s 6ms/step - loss: 0.3045 - accuracy: 0.8357 - val_loss: 0.2574 - val_accuracy: 0.9000
Epoch 176/400
9/9 [==============================] - 0s 6ms/step - loss: 0.3038 - accuracy: 0.8357 - val_loss: 0.2571 - val_accuracy: 0.9000
Epoch 177/400
9/9 [==============================] - 0s 6ms/step - loss: 0.3032 - accuracy: 0.8357 - val_loss: 0.2567 - val_accuracy: 0.9000
Epoch 178/400
9/9 [==============================] - 0s 5ms/step - loss: 0.3024 - accuracy: 0.8357 - val_loss: 0.2561 - val_accuracy: 0.9000
Epoch 179/400
9/9 [==============================] - 0s 12ms/step - loss: 0.3017 - accuracy: 0.8357 - val_loss: 0.2558 - val_accuracy: 0.9000
Epoch 180/400
9/9 [==============================] - 0s 6ms/step - loss: 0.3010 - accuracy: 0.8357 - val_loss: 0.2554 - val_accuracy: 0.9000
Epoch 181/400
9/9 [==============================] - 0s 10ms/step - loss: 0.3004 - accuracy: 0.8357 - val_loss: 0.2550 - val_accuracy: 0.9000
Epoch 182/400
9/9 [======

9/9 [==============================] - 0s 6ms/step - loss: 0.2753 - accuracy: 0.8357 - val_loss: 0.2393 - val_accuracy: 0.9000
Epoch 234/400
9/9 [==============================] - 0s 5ms/step - loss: 0.2747 - accuracy: 0.8357 - val_loss: 0.2391 - val_accuracy: 0.9000
Epoch 235/400
9/9 [==============================] - 0s 5ms/step - loss: 0.2743 - accuracy: 0.8357 - val_loss: 0.2389 - val_accuracy: 0.9000
Epoch 236/400
9/9 [==============================] - 0s 9ms/step - loss: 0.2739 - accuracy: 0.8357 - val_loss: 0.2388 - val_accuracy: 0.9000
Epoch 237/400
9/9 [==============================] - 0s 6ms/step - loss: 0.2736 - accuracy: 0.8357 - val_loss: 0.2384 - val_accuracy: 0.9000
Epoch 238/400
9/9 [==============================] - 0s 10ms/step - loss: 0.2734 - accuracy: 0.8357 - val_loss: 0.2381 - val_accuracy: 0.9000
Epoch 239/400
9/9 [==============================] - 0s 10ms/step - loss: 0.2729 - accuracy: 0.8357 - val_loss: 0.2382 - val_accuracy: 0.9000
Epoch 240/400
9/9 [======

9/9 [==============================] - 0s 4ms/step - loss: 0.2564 - accuracy: 0.8357 - val_loss: 0.2276 - val_accuracy: 0.9000
Epoch 292/400
9/9 [==============================] - 0s 4ms/step - loss: 0.2560 - accuracy: 0.8357 - val_loss: 0.2274 - val_accuracy: 0.9000
Epoch 293/400
9/9 [==============================] - 0s 4ms/step - loss: 0.2561 - accuracy: 0.8357 - val_loss: 0.2266 - val_accuracy: 0.9000
Epoch 294/400
9/9 [==============================] - 0s 4ms/step - loss: 0.2555 - accuracy: 0.8357 - val_loss: 0.2263 - val_accuracy: 0.9000
Epoch 295/400
9/9 [==============================] - 0s 4ms/step - loss: 0.2553 - accuracy: 0.8357 - val_loss: 0.2264 - val_accuracy: 0.9000
Epoch 296/400
9/9 [==============================] - 0s 3ms/step - loss: 0.2550 - accuracy: 0.8357 - val_loss: 0.2263 - val_accuracy: 0.9000
Epoch 297/400
9/9 [==============================] - 0s 4ms/step - loss: 0.2549 - accuracy: 0.8357 - val_loss: 0.2260 - val_accuracy: 0.9000
Epoch 298/400
9/9 [========

9/9 [==============================] - 0s 4ms/step - loss: 0.2418 - accuracy: 0.8857 - val_loss: 0.2129 - val_accuracy: 0.9000
Epoch 350/400
9/9 [==============================] - 0s 4ms/step - loss: 0.2414 - accuracy: 0.8857 - val_loss: 0.2127 - val_accuracy: 0.9000
Epoch 351/400
9/9 [==============================] - 0s 7ms/step - loss: 0.2412 - accuracy: 0.8857 - val_loss: 0.2123 - val_accuracy: 0.9000
Epoch 352/400
9/9 [==============================] - 0s 4ms/step - loss: 0.2408 - accuracy: 0.8857 - val_loss: 0.2125 - val_accuracy: 0.9000
Epoch 353/400
9/9 [==============================] - 0s 6ms/step - loss: 0.2407 - accuracy: 0.8857 - val_loss: 0.2125 - val_accuracy: 0.9000
Epoch 354/400
9/9 [==============================] - 0s 20ms/step - loss: 0.2402 - accuracy: 0.8857 - val_loss: 0.2124 - val_accuracy: 0.9000
Epoch 355/400
9/9 [==============================] - 0s 8ms/step - loss: 0.2403 - accuracy: 0.8857 - val_loss: 0.2121 - val_accuracy: 0.9000
Epoch 356/400
9/9 [=======

In [11]:
predictions = model.predict(scaled_X_test)

In [12]:
preds = []
for x in predictions:
    if x >= 0.5:
        preds.append(1)
    else:
        preds.append(0)

In [13]:
from sklearn.metrics import classification_report,confusion_matrix

In [14]:
print(classification_report(y1_test,preds))

              precision    recall  f1-score   support

         0.0       0.95      0.94      0.94       108
         1.0       0.50      0.58      0.54        12

    accuracy                           0.90       120
   macro avg       0.73      0.76      0.74       120
weighted avg       0.91      0.90      0.90       120

